In [1]:
import requests
import json

In [2]:
dati_comuni=requests.get('https://raw.githubusercontent.com/MatteoHenryChinaski/Comuni-Italiani-2018-Sql-Json-excel/master/italy_cities.json').json()
comuniGeojson=requests.get('https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_municipalities.geojson').json()

In [3]:
dati_province=requests.get('https://raw.githubusercontent.com/MatteoHenryChinaski/Comuni-Italiani-2018-Sql-Json-excel/master/italy_provincies.json').json()
provinceGeojson=requests.get('https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson').json()

In [7]:
covidProvincie=requests.get('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-province-latest.json').json()

In [8]:
dati_comuni[0],dati_province[0]

({'istat': '1001',
  'comune': 'Agliè',
  'regione': 'Piemonte',
  'provincia': 'TO',
  'prefisso': '124',
  'cod_fisco': 'A074',
  'superficie': '13.28',
  'num_residenti': '2669',
  'density': 0.001183739000851547},
 {'sigla': 'AG',
  'provincia': 'Agrigento',
  'superficie': '3044.85',
  'residenti': '446081',
  'num_comuni': '43',
  'id_regione': '15'})

In [9]:
for i,comune in enumerate(dati_comuni[:-2]):
    prov_code=comune['provincia']
    prov_pop=0
    for provincia in dati_province[:-2]:
        if provincia['sigla']==prov_code:
            prov_pop=float(provincia['residenti'])
            dati_comuni[i]['density']=float(comune['num_residenti'])/prov_pop




0.001183739000851547

In [11]:
for i,comune in enumerate(dati_comuni[:-2]):
    prov_code=comune['provincia']
    prov_pop=0
    for provincia in covidProvincie:
        if provincia['sigla_provincia']==prov_code:
            prov_covid=float(provincia['totale_casi'])
            dati_comuni[i]['covid']=dati_comuni[i]['density']*prov_covid

In [12]:
dati_comuni[0]

{'istat': '1001',
 'comune': 'Agliè',
 'regione': 'Piemonte',
 'provincia': 'TO',
 'prefisso': '124',
 'cod_fisco': 'A074',
 'superficie': '13.28',
 'num_residenti': '2669',
 'density': 0.001183739000851547,
 'covid': 61.481036226227644}

In [8]:
covidProvincie=requests.get('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-province-latest.json').json()

In [13]:
comuniGeojson['features'][0]["properties"]

{'name': 'Agliè',
 'op_id': '130',
 'minint_elettorale': '1010810010',
 'prov_name': 'Torino',
 'prov_istat_code': '001',
 'prov_istat_code_num': 1,
 'prov_acr': 'TO',
 'reg_name': 'Piemonte',
 'reg_istat_code': '01',
 'reg_istat_code_num': 1,
 'opdm_id': '1',
 'com_catasto_code': 'A074',
 'com_istat_code': '001001',
 'com_istat_code_num': 1001}

In [25]:
for i,comGeo in enumerate(comuniGeojson['features'][:3]):
    for comune in dati_comuni[:-2]:
        if str(comGeo['properties']['com_catasto_code'])==comune['cod_fisco']:
            comuniGeojson['features'][i]['properties']['covid']=comune['covid']

In [26]:
comuniGeojson["features"][0]['properties']

{'name': 'Agliè',
 'op_id': '130',
 'minint_elettorale': '1010810010',
 'prov_name': 'Torino',
 'prov_istat_code': '001',
 'prov_istat_code_num': 1,
 'prov_acr': 'TO',
 'reg_name': 'Piemonte',
 'reg_istat_code': '01',
 'reg_istat_code_num': 1,
 'opdm_id': '1',
 'com_catasto_code': 'A074',
 'com_istat_code': '001001',
 'com_istat_code_num': 1001,
 'covid': 61.481036226227644}

In [27]:
json.dump(comuniGeojson,open("comuniCovidStat.json",'w'))